In [1]:
import sqlite3
import pandas as pd
# import modin.pandas as pd
import numpy as np
# random seed for numpy --> random seed for sklearn, pandas, statsmodels
np.random.seed(99)

import random
random.seed(99)

#### SQL Statements 
- EVENT_OUTCOMES --> EVENT_OUTCOMES_MONTHLY
    - create table EVENT_OUTCOMES_MONTHLY AS select *, SUM(FATALITIES), SUM(COUNT_PRO), SUM(COUNT_SRZ), SUM(COUNT_STRA), SUM(COUNT) AS NUMBER_EVENTS, SUM(FATALITIES_PRO), SUM(FATALITIES_SRZ), SUM(FATALITIES_STRA) from TB001_EVENT_OUTCOMES GROUP BY GID_0, EVENT_DATE_MONTH 

- EVENT_OUTCOMES_MONTHLY --> EVENT_OUTCOMES_MONTHLY_START_DATE
    - create TABLE TB003_EVENT_OUTCOMES_MONTHLY_START_DATE as select DISTINCT TB002_EVENT_OUTCOMES_MONTHLY.*, UNREGION1, UNREGION2, START_DATE FROM TB002_EVENT_OUTCOMES_MONTHLY, TB001_CARTESIAN_FRAME WHERE TB002_EVENT_OUTCOMES_MONTHLY.GID_0=TB001_CARTESIAN_FRAME.GID_0 AND T  B002_EVENT_OUTCOMES_MONTHLY.GID_1=TB001_CARTESIAN_FRAME.GID_1

In [2]:
# conn = sqlite3.connect("../data/01_ACLED.db")
# acled_monthly_adm1 = pd.read_sql_query("""
#     SELECT * FROM TB003_EVENT_OUTCOMES_MONTHLY_START_DATE ORDER BY GID_0, GID_1, EVENT_DATE_MONTH;
#     """, conn)
# acled_monthly_adm1.to_csv("../data/TB003_EVENT_OUTCOMES_MONTHLY_START_DATE.csv", index=False)
# acled_monthly_adm1

In [3]:
acled_monthly_adm1 = pd.read_csv("../data/TB003_EVENT_OUTCOMES_MONTHLY_START_DATE.csv")
# acled_monthly_adm1[acled_monthly_adm1["GID_0"] == "AFG"].sort_values(by="EVENT_DATE")
acled_monthly_adm1

,GID_1,GID_0,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE,EVENT_DATE,EVENT_DATE_MONTH,EVENT_DATE_QUARTER,EVENT_DATE_YEAR,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),NUMBER_EVENTS,SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA)
0,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2019-09-27 00:00:00,2019-09-01,2019-07-01,2019,0,3,0,0,3,0,0,0
1,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-03-17 00:00:00,2020-03-01,2020-01-01,2020,0,1,0,1,2,0,0,0
2,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-05-20 00:00:00,2020-05-01,2020-04-01,2020,0,1,0,0,1,0,0,0
3,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-07-03 00:00:00,2020-07-01,2020-07-01,2020,0,1,0,0,1,0,0,0
4,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-08-25 00:00:00,2020-08-01,2020-07-01,2020,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16949,ZWE.9_1,ZWE,Eastern Africa,Africa,-21.2000,27.8500,1997-01-01,2019-03-25 00:00:00,2019-03-01,2019-01-01,2019,0,2,1,0,3,0,0,0
16950,ZWE.9_1,ZWE,Eastern Africa,Africa,-22.2167,29.9833,1997-01-01,2019-07-27 00:00:00,2019-07-01,2019-07-01,2019,1,6,1,0,7,0,1,0
16951,ZWE.9_1,ZWE,Eastern Africa,Africa,-20.9333,29.0000,1997-01-01,2020-07-31 00:00:00,2020-07-01,2020-07-01,2020,3,12,6,1,19,2,1,0
16952,ZWE.9_1,ZWE,Eastern Africa,Africa,-22.2233,29.9839,1997-01-01,2020-11-19 00:00:00,2020-11-01,2020-10-01,2020,5,4,1,1,6,4,1,0


### Monthly ACLED on ADM0 Level

In [4]:
agg_dict = {'SUM(FATALITIES)': np.sum, 'SUM(COUNT_PRO)': np.sum, 'SUM(COUNT_SRZ)': np.sum, 'SUM(COUNT_STRA)': np.sum, 'SUM(FATALITIES_PRO)': np.sum, 'SUM(FATALITIES_SRZ)': np.sum, 'SUM(FATALITIES_STRA)': np.sum, "NUMBER_EVENTS": np.sum}
acled_monthly_adm0_time_series = acled_monthly_adm1.groupby(by=["GID_0", "EVENT_DATE_MONTH", "UNREGION1", "UNREGION2", "LATITUDE", "LONGITUDE", "START_DATE"]).agg(agg_dict)
acled_monthly_adm0_time_series = acled_monthly_adm0_time_series.reset_index()
acled_monthly_adm0_time_series.head()

,GID_0,EVENT_DATE_MONTH,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA),NUMBER_EVENTS
0,ABW,2019-09-01,Caribbean,Americas,12.524,-70.027,2019-01-01,0,3,0,0,0,0,0,3
1,ABW,2020-03-01,Caribbean,Americas,12.524,-70.027,2019-01-01,0,1,0,1,0,0,0,2
2,ABW,2020-05-01,Caribbean,Americas,12.524,-70.027,2019-01-01,0,1,0,0,0,0,0,1
3,ABW,2020-07-01,Caribbean,Americas,12.524,-70.027,2019-01-01,0,1,0,0,0,0,0,1
4,ABW,2020-08-01,Caribbean,Americas,12.524,-70.027,2019-01-01,0,1,0,0,0,0,0,1


##### add missing months

In [5]:
from datetime import datetime
from dateutil import rrule

acled_monthly_adm0_time_series["START_DATE"]= pd.to_datetime(acled_monthly_adm0_time_series["START_DATE"])
acled_monthly_adm0_time_series["EVENT_DATE_MONTH"]= pd.to_datetime(acled_monthly_adm0_time_series["EVENT_DATE_MONTH"])

# insert a row for country
gid0 = acled_monthly_adm0_time_series['GID_0'].unique()
for g in gid0:
    print(g)
    # start and end dates
    start_date = acled_monthly_adm0_time_series[acled_monthly_adm0_time_series["GID_0"] == g]["START_DATE"].iloc[0]
    unregion1 = acled_monthly_adm0_time_series[acled_monthly_adm0_time_series["GID_0"] == g]["UNREGION1"].iloc[0]
    unregion2 = acled_monthly_adm0_time_series[acled_monthly_adm0_time_series["GID_0"] == g]["UNREGION2"].iloc[0]
    lat = acled_monthly_adm0_time_series[acled_monthly_adm0_time_series["GID_0"] == g]["LATITUDE"].iloc[0]
    long = acled_monthly_adm0_time_series[acled_monthly_adm0_time_series["GID_0"] == g]["LONGITUDE"].iloc[0]

    # start_date = datetime.strptime(start_date + " 00:00:00", "%Y-%m-%d %H:%M:%S")
    # print(start_date)
    
    end_date = datetime(2022, 2, 28)
    # print(end_date)
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=start_date, until=end_date):
        # print(dt)
        if not ((acled_monthly_adm0_time_series['GID_0'] == g) & (acled_monthly_adm0_time_series['EVENT_DATE_MONTH'] == dt)).any():
            new_row = pd.DataFrame([{'GID_0': g, 'EVENT_DATE_MONTH': dt, 'LATITUDE': lat, "LONGITUDE": long, "START_DATE":start_date, "UNREGION1": unregion1, "UNREGION2": unregion2}])
            # print(new_row)
            acled_monthly_adm0_time_series = pd.concat([acled_monthly_adm0_time_series, new_row])

ABW
AFG
AGO
AIA
ALA
ALB
AND
ARE
ARG
ARM
ASM
ATA
ATG
AUS
AUT
AZE
BDI
BEL
BEN
BES
BFA
BGD
BGR
BHR
BHS
BIH
BLM
BLR
BLZ
BMU
BOL
BRA
BRB
BRN
BTN
BWA
CAF
CAN
CHE
CHL
CHN
CIV
CMR
COD
COG
COK
COL
COM
CPV
CRI
CUB
CUW
CYM
CYP
CZE
DEU
DJI
DMA
DNK
DOM
DZA
ECU
EGY
ERI
ESH
ESP
EST
ETH
FIN
FJI
FLK
FRA
FRO
FSM
GAB
GBR
GEO
GGY
GHA
GIB
GIN
GLP
GMB
GNB
GNQ
GRC
GRD
GRL
GTM
GUF
GUM
GUY
HKG
HND
HRV
HTI
HUN
IDN
IMN
IND
IRL
IRN
IRQ
ISL
ISR
ITA
JAM
JEY
JOR
JPN
KAZ
KEN
KGZ
KHM
KNA
KOR
KWT
LAO
LBN
LBR
LBY
LCA
LIE
LKA
LSO
LTU
LUX
LVA
MAC
MAR
MCO
MDA
MDG
MDV
MEX
MKD
MLI
MLT
MMR
MNE
MNG
MNP
MOZ
MRT
MSR
MTQ
MUS
MWI
MYS
MYT
NAM
NCL
NER
NGA
NIC
NLD
NOR
NPL
NZL
OMN
PAK
PAN
PER
PHL
PNG
POL
PRI
PRK
PRT
PRY
PSE
PYF
QAT
REU
ROU
RUS
RWA
SAU
SDN
SEN
SGP
SHN
SJM
SLB
SLE
SLV
SMR
SOM
SRB
SSD
STP
SUR
SVK
SVN
SWE
SWZ
SXM
SYC
SYR
TCA
TCD
TGO
THA
TJK
TKM
TLS
TON
TTO
TUN
TUR
TWN
TZA
UGA
UKR
URY
USA
UZB
VAT
VCT
VEN
VIR
VNM
VUT
WLF
WSM
XAR
XIT
XKK
XKO
XNC
YEM
ZAF
ZMB
ZWE


In [6]:
acled_monthly_adm0_time_series = acled_monthly_adm0_time_series.sort_values(by=["GID_0", "EVENT_DATE_MONTH"])
acled_monthly_adm0_time_series[["SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)"]] = acled_monthly_adm0_time_series[["SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)"]].fillna(0.0)

acled_monthly_adm0_time_series

,GID_0,EVENT_DATE_MONTH,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA),NUMBER_EVENTS
0,ABW,2019-01-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-02-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-03-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-04-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-05-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16949,ZWE,2021-10-01,Eastern Africa,Africa,-17.3500,31.0667,1997-01-01,4.0,28.0,12.0,1.0,3.0,1.0,0.0,41.0
16950,ZWE,2021-11-01,Eastern Africa,Africa,-17.8277,31.0534,1997-01-01,1.0,12.0,8.0,0.0,1.0,0.0,0.0,20.0
16951,ZWE,2021-12-01,Eastern Africa,Africa,-18.9166,29.8167,1997-01-01,6.0,12.0,4.0,3.0,1.0,5.0,0.0,19.0
16952,ZWE,2022-01-01,Eastern Africa,Africa,-17.8277,31.0534,1997-01-01,2.0,7.0,5.0,2.0,2.0,0.0,0.0,14.0


In [7]:
# acled_monthly_adm0.to_sql(name='TB005_ACLED_MONTHLY_ADM0_TIME_SERIES', con=conn)
# conn.close()

In [8]:
acled_monthly_adm0_time_series.to_csv("../data/TB005_ACLED_MONTHLY_ADM0_TIME_SERIES.csv", index=False)

In [9]:
acled_monthly_adm0_time_series

,GID_0,EVENT_DATE_MONTH,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA),NUMBER_EVENTS
0,ABW,2019-01-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-02-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-03-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-04-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,ABW,2019-05-01,Caribbean,Americas,12.5240,-70.0270,2019-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16949,ZWE,2021-10-01,Eastern Africa,Africa,-17.3500,31.0667,1997-01-01,4.0,28.0,12.0,1.0,3.0,1.0,0.0,41.0
16950,ZWE,2021-11-01,Eastern Africa,Africa,-17.8277,31.0534,1997-01-01,1.0,12.0,8.0,0.0,1.0,0.0,0.0,20.0
16951,ZWE,2021-12-01,Eastern Africa,Africa,-18.9166,29.8167,1997-01-01,6.0,12.0,4.0,3.0,1.0,5.0,0.0,19.0
16952,ZWE,2022-01-01,Eastern Africa,Africa,-17.8277,31.0534,1997-01-01,2.0,7.0,5.0,2.0,2.0,0.0,0.0,14.0


### Monthly ACLED on ADM1 Level (Provinces)

In [10]:
import sqlite3
import pandas as pd
import numpy as np

In [11]:
# conn = sqlite3.connect("../data/01_ACLED.db")
# acled_monthly_adm1 = pd.read_sql_query("""
#     SELECT * FROM TB003_EVENT_OUTCOMES_MONTHLY_START_DATE ORDER BY GID_0, GID_1, EVENT_DATE_MONTH;
#     """, conn)
# acled_monthly_adm1.to_csv("../data/TB003_EVENT_OUTCOMES_MONTHLY_START_DATE.csv", index=False)
acled_monthly_adm1 = pd.read_csv("../data/TB003_EVENT_OUTCOMES_MONTHLY_START_DATE.csv")
acled_monthly_adm1

,GID_1,GID_0,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE,EVENT_DATE,EVENT_DATE_MONTH,EVENT_DATE_QUARTER,EVENT_DATE_YEAR,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),NUMBER_EVENTS,SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA)
0,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2019-09-27 00:00:00,2019-09-01,2019-07-01,2019,0,3,0,0,3,0,0,0
1,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-03-17 00:00:00,2020-03-01,2020-01-01,2020,0,1,0,1,2,0,0,0
2,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-05-20 00:00:00,2020-05-01,2020-04-01,2020,0,1,0,0,1,0,0,0
3,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-07-03 00:00:00,2020-07-01,2020-07-01,2020,0,1,0,0,1,0,0,0
4,ABW.1_1,ABW,Caribbean,Americas,12.5240,-70.0270,2019-01-01,2020-08-25 00:00:00,2020-08-01,2020-07-01,2020,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16949,ZWE.9_1,ZWE,Eastern Africa,Africa,-21.2000,27.8500,1997-01-01,2019-03-25 00:00:00,2019-03-01,2019-01-01,2019,0,2,1,0,3,0,0,0
16950,ZWE.9_1,ZWE,Eastern Africa,Africa,-22.2167,29.9833,1997-01-01,2019-07-27 00:00:00,2019-07-01,2019-07-01,2019,1,6,1,0,7,0,1,0
16951,ZWE.9_1,ZWE,Eastern Africa,Africa,-20.9333,29.0000,1997-01-01,2020-07-31 00:00:00,2020-07-01,2020-07-01,2020,3,12,6,1,19,2,1,0
16952,ZWE.9_1,ZWE,Eastern Africa,Africa,-22.2233,29.9839,1997-01-01,2020-11-19 00:00:00,2020-11-01,2020-10-01,2020,5,4,1,1,6,4,1,0


In [12]:
# format columns to datetime format
acled_monthly_adm1["START_DATE"]= pd.to_datetime(acled_monthly_adm1["START_DATE"])
acled_monthly_adm1["EVENT_DATE_MONTH"]= pd.to_datetime(acled_monthly_adm1["EVENT_DATE_MONTH"])

acled_monthly_adm1_time_series = list()

# insert a row for country
gid1 = acled_monthly_adm1['GID_1'].unique()
for g in gid1:
    # constant variables for each country
    start_date = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["START_DATE"].iloc[0]
    unregion1 = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["UNREGION1"].iloc[0]
    unregion2 = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["UNREGION2"].iloc[0]
    lat = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["LATITUDE"].iloc[0]
    long = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["LONGITUDE"].iloc[0]
    gid0 = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g]["GID_0"].iloc[0]

    print(g)
    sampled = acled_monthly_adm1[acled_monthly_adm1["GID_1"] == g][["EVENT_DATE_MONTH", "SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)"]]
    # print(sampled)
    sampled = sampled.set_index(pd.DatetimeIndex(sampled["EVENT_DATE_MONTH"]))
    sampled = sampled.resample("MS").sum()
    sampled["EVENT_DATE_MONTH"] = sampled.index
    sampled[["GID_0", "GID_1", "UNREGION1", "UNREGION2", "LATITUDE", "LONGITUDE", "START_DATE"]] = [gid0, g, unregion1, unregion2, lat, long, start_date]
    
    # print(sampled.values.tolist())
    # print(sampled)

    acled_monthly_adm1_time_series.extend(sampled.values.tolist())
    # print(acled_monthly_adm1_time_series)

# turn list into df
acled_monthly_adm1_time_series = pd.DataFrame.from_records(acled_monthly_adm1_time_series, columns=["SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)", "EVENT_DATE_MONTH", "GID_0", "GID_1", "UNREGION1", "UNREGION2", "LATITUDE", "LONGITUDE", "START_DATE"])
# print(acled_monthly_adm1_time_series)
acled_monthly_adm1_time_series = acled_monthly_adm1_time_series.reset_index(drop=True)
acled_monthly_adm1_time_series

ABW.1_1
AFG.10_1
AFG.11_1
AFG.12_1
AFG.13_1
AFG.14_1
AFG.15_1
AFG.16_1
AFG.17_1
AFG.18_1
AFG.19_1
AFG.1_1
AFG.20_1
AFG.21_1
AFG.22_1
AFG.25_1
AFG.26_1
AFG.28_1
AFG.31_1
AFG.33_1
AFG.34_1
AFG.4_1
AFG.6_1
AFG.7_1
AFG.8_1
AFG.9_1
AGO.10_1
AGO.11_1
AGO.12_1
AGO.13_1
AGO.14_1
AGO.15_1
AGO.16_1
AGO.17_1
AGO.18_1
AGO.1_1
AGO.2_1
AGO.3_1
AGO.4_1
AGO.5_1
AGO.6_1
AGO.7_1
AGO.8_1
AGO.9_1
AIA.1_1
ALA.12_1
ALB.10_1
ALB.11_1
ALB.12_1
ALB.1_1
ALB.2_1
ALB.3_1
ALB.4_1
ALB.5_1
ALB.7_1
ALB.8_1
ALB.9_1
AND.1_1
AND.2_1
ARE.1_1
ARE.3_1
ARG.11_1
ARG.13_1
ARG.15_1
ARG.16_1
ARG.17_1
ARG.18_1
ARG.19_1
ARG.1_1
ARG.20_1
ARG.21_1
ARG.22_1
ARG.24_1
ARG.2_1
ARG.3_1
ARG.4_1
ARG.5_1
ARG.6_1
ARG.7_1
ARM.10_1
ARM.11_1
ARM.1_1
ARM.2_1
ARM.3_1
ARM.4_1
ARM.5_1
ARM.7_1
ARM.8_1
ARM.9_1
ASM.1_1
ASM.4_1
ATA.1_1
ATG.3_1
ATG.4_1
ATG.5_1
ATG.7_1
AUS.10_1
AUS.2_1
AUS.5_1
AUS.7_1
AUS.8_1
AUT.3_1
AUT.5_1
AUT.6_1
AUT.7_1
AUT.8_1
AUT.9_1
AZE.10_1
AZE.1_1
AZE.2_1
AZE.4_1
AZE.5_1
AZE.6_1
AZE.9_1
BDI.10_1
BDI.11_1
BDI.12_1
BDI.13_1
BDI.1

,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),NUMBER_EVENTS,SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA),EVENT_DATE_MONTH,GID_0,GID_1,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE
0,0,3,0,0,3,0,0,0,2019-09-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
1,0,0,0,0,0,0,0,0,2019-10-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
2,0,0,0,0,0,0,0,0,2019-11-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
3,0,0,0,0,0,0,0,0,2019-12-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
4,0,0,0,0,0,0,0,0,2020-01-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161953,0,0,0,0,0,0,0,0,2020-08-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161954,0,0,0,0,0,0,0,0,2020-09-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161955,0,0,0,0,0,0,0,0,2020-10-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161956,5,4,1,1,6,4,1,0,2020-11-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01


In [13]:
acled_monthly_adm1_time_series = acled_monthly_adm1_time_series.sort_values(by=["GID_1", "EVENT_DATE_MONTH"])
acled_monthly_adm1_time_series[["SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)"]] = acled_monthly_adm1_time_series[["SUM(FATALITIES)", "SUM(COUNT_PRO)", "SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)"]].fillna(0.0)

acled_monthly_adm1_time_series

,SUM(FATALITIES),SUM(COUNT_PRO),SUM(COUNT_SRZ),SUM(COUNT_STRA),NUMBER_EVENTS,SUM(FATALITIES_PRO),SUM(FATALITIES_SRZ),SUM(FATALITIES_STRA),EVENT_DATE_MONTH,GID_0,GID_1,UNREGION1,UNREGION2,LATITUDE,LONGITUDE,START_DATE
0,0,3,0,0,3,0,0,0,2019-09-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
1,0,0,0,0,0,0,0,0,2019-10-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
2,0,0,0,0,0,0,0,0,2019-11-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
3,0,0,0,0,0,0,0,0,2019-12-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
4,0,0,0,0,0,0,0,0,2020-01-01,ABW,ABW.1_1,Caribbean,Americas,12.5240,-70.027,2019-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161953,0,0,0,0,0,0,0,0,2020-08-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161954,0,0,0,0,0,0,0,0,2020-09-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161955,0,0,0,0,0,0,0,0,2020-10-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01
161956,5,4,1,1,6,4,1,0,2020-11-01,ZWE,ZWE.9_1,Eastern Africa,Africa,-19.7833,29.200,1997-01-01


In [14]:
acled_monthly_adm1_time_series.to_csv("../data/TB004_ACLED_MONTHLY_ADM1_TIME_SERIES.csv", index=True)


In [15]:
# acled_monthly_adm1_time_series.to_sql(name='TB004_ACLED_MONTHLY_ADM1_TIME_SERIES', con=conn)
# conn.close()


### Log-Transformation
- to allow log change caluclation in prediction

#### ADM0 MONTHLY

In [16]:
# pivot df to create column for every country fatality time series
pivot_columns = ["SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", 
                "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)", "GID_0", "EVENT_DATE_MONTH", 
                "SUM(FATALITIES)", "SUM(COUNT_PRO)"]
acled_monthly_adm0_pivot = acled_monthly_adm0_time_series[pivot_columns]
acled_monthly_adm0_pivot = pd.pivot_table(acled_monthly_adm0_pivot, columns=["GID_0"], index=["EVENT_DATE_MONTH"])
acled_monthly_adm0_pivot.columns = acled_monthly_adm0_pivot.columns.map('{0[0]}_{0[1]}'.format)
acled_monthly_adm0_pivot

,NUMBER_EVENTS_ABW,NUMBER_EVENTS_AFG,NUMBER_EVENTS_AGO,NUMBER_EVENTS_AIA,NUMBER_EVENTS_ALA,NUMBER_EVENTS_ALB,NUMBER_EVENTS_AND,NUMBER_EVENTS_ARE,NUMBER_EVENTS_ARG,NUMBER_EVENTS_ARM,...,SUM(FATALITIES_STRA)_WSM,SUM(FATALITIES_STRA)_XAR,SUM(FATALITIES_STRA)_XIT,SUM(FATALITIES_STRA)_XKK,SUM(FATALITIES_STRA)_XKO,SUM(FATALITIES_STRA)_XNC,SUM(FATALITIES_STRA)_YEM,SUM(FATALITIES_STRA)_ZAF,SUM(FATALITIES_STRA)_ZMB,SUM(FATALITIES_STRA)_ZWE
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-02-01,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-03-01,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-04-01,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-05-01,NaN,NaN,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,0.0,171.0,28.0,0.0,0.0,19.0,0.0,0.0,264.0,21.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-11-01,2.0,168.0,5.0,0.0,1.0,12.0,1.0,0.0,247.0,39.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2021-12-01,1.0,184.0,20.0,0.0,1.0,6.0,0.0,0.0,212.0,36.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0


In [17]:
acled_monthly_adm0_log = np.log(acled_monthly_adm0_pivot+1)
# Calculates the difference of a Dataframe element compared with another element in the Dataframe (default is element in previous row).
acled_monthly_adm0_log_change = acled_monthly_adm0_log#.diff() # first row set to zero
# drop first row
# acled_monthly_adm0_log_change = acled_monthly_adm0_log_change.iloc[1:]
acled_monthly_adm0_log

,NUMBER_EVENTS_ABW,NUMBER_EVENTS_AFG,NUMBER_EVENTS_AGO,NUMBER_EVENTS_AIA,NUMBER_EVENTS_ALA,NUMBER_EVENTS_ALB,NUMBER_EVENTS_AND,NUMBER_EVENTS_ARE,NUMBER_EVENTS_ARG,NUMBER_EVENTS_ARM,...,SUM(FATALITIES_STRA)_WSM,SUM(FATALITIES_STRA)_XAR,SUM(FATALITIES_STRA)_XIT,SUM(FATALITIES_STRA)_XKK,SUM(FATALITIES_STRA)_XKO,SUM(FATALITIES_STRA)_XNC,SUM(FATALITIES_STRA)_YEM,SUM(FATALITIES_STRA)_ZAF,SUM(FATALITIES_STRA)_ZMB,SUM(FATALITIES_STRA)_ZWE
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,1.609438,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-02-01,NaN,NaN,2.772589,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-03-01,NaN,NaN,1.791759,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-04-01,NaN,NaN,1.386294,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-05-01,NaN,NaN,3.496508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,0.000000,5.147494,3.367296,0.0,0.000000,2.995732,0.000000,0.000000,5.579730,3.091042,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2021-11-01,1.098612,5.129899,1.791759,0.0,0.693147,2.564949,0.693147,0.000000,5.513429,3.688879,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2021-12-01,0.693147,5.220356,3.044522,0.0,0.693147,1.945910,0.000000,0.000000,5.361292,3.610918,...,0.0,0.0,0.0,0.0,0.0,0.0,2.397895,0.0,0.0,0.0


In [18]:
# acled_monthly_adm0_log_change[["SUM(FATALITIES)_IND"]].loc["2012-10-01":]

In [19]:
acled_monthly_adm0_log.to_csv("../data/TB011_ACLED_MONTHLY_ADM0_LOG.csv", index=True)

#### ADM1 MONTHLY

In [20]:
# pivot df to create column for every country fatality time series
pivot_columns = ["SUM(COUNT_SRZ)", "SUM(COUNT_STRA)", "NUMBER_EVENTS", 
                "SUM(FATALITIES_PRO)", "SUM(FATALITIES_SRZ)", "SUM(FATALITIES_STRA)", "GID_1", "EVENT_DATE_MONTH", 
                "SUM(FATALITIES)", "SUM(COUNT_PRO)"]
acled_monthly_adm1_pivot = acled_monthly_adm1_time_series[pivot_columns]
acled_monthly_adm1_pivot = pd.pivot_table(acled_monthly_adm1_pivot, columns=["GID_1"], index=["EVENT_DATE_MONTH"])
acled_monthly_adm1_pivot.columns = acled_monthly_adm1_pivot.columns.map('{0[0]}_{0[1]}'.format)
acled_monthly_adm1_pivot

,NUMBER_EVENTS_ABW.1_1,NUMBER_EVENTS_AFG.10_1,NUMBER_EVENTS_AFG.11_1,NUMBER_EVENTS_AFG.12_1,NUMBER_EVENTS_AFG.13_1,NUMBER_EVENTS_AFG.14_1,NUMBER_EVENTS_AFG.15_1,NUMBER_EVENTS_AFG.16_1,NUMBER_EVENTS_AFG.17_1,NUMBER_EVENTS_AFG.18_1,...,SUM(FATALITIES_STRA)_ZWE.10_1,SUM(FATALITIES_STRA)_ZWE.1_1,SUM(FATALITIES_STRA)_ZWE.2_1,SUM(FATALITIES_STRA)_ZWE.3_1,SUM(FATALITIES_STRA)_ZWE.4_1,SUM(FATALITIES_STRA)_ZWE.5_1,SUM(FATALITIES_STRA)_ZWE.6_1,SUM(FATALITIES_STRA)_ZWE.7_1,SUM(FATALITIES_STRA)_ZWE.8_1,SUM(FATALITIES_STRA)_ZWE.9_1
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2021-11-01,2.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-01,1.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# calculate log change relative to past period for each column

acled_monthly_adm1_log = np.log(acled_monthly_adm1_pivot+1)
# Calculates the difference of a Dataframe element compared with another element in the Dataframe (default is element in previous row).
# acled_monthly_adm1_log_change = acled_monthly_adm1_log #.diff() # first row set to zero

# drop first row
# acled_monthly_adm1_log_change = acled_monthly_adm1_log_change.iloc[1:]

acled_monthly_adm1_log

,NUMBER_EVENTS_ABW.1_1,NUMBER_EVENTS_AFG.10_1,NUMBER_EVENTS_AFG.11_1,NUMBER_EVENTS_AFG.12_1,NUMBER_EVENTS_AFG.13_1,NUMBER_EVENTS_AFG.14_1,NUMBER_EVENTS_AFG.15_1,NUMBER_EVENTS_AFG.16_1,NUMBER_EVENTS_AFG.17_1,NUMBER_EVENTS_AFG.18_1,...,SUM(FATALITIES_STRA)_ZWE.10_1,SUM(FATALITIES_STRA)_ZWE.1_1,SUM(FATALITIES_STRA)_ZWE.2_1,SUM(FATALITIES_STRA)_ZWE.3_1,SUM(FATALITIES_STRA)_ZWE.4_1,SUM(FATALITIES_STRA)_ZWE.5_1,SUM(FATALITIES_STRA)_ZWE.6_1,SUM(FATALITIES_STRA)_ZWE.7_1,SUM(FATALITIES_STRA)_ZWE.8_1,SUM(FATALITIES_STRA)_ZWE.9_1
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-10-01,0.000000,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2021-11-01,1.098612,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-01,0.693147,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
acled_monthly_adm1_log.to_csv("../data/TB012_ACLED_MONTHLY_ADM1_LOG.csv", index=True)

### ADM0 QUARTERLY

In [23]:
acled_quarterly_adm0_pivot = acled_monthly_adm0_pivot.resample('3MS').apply(lambda x: x.sum(skipna=False))
# log all values to enable later lognchange calculation by difference
acled_quarterly_adm0_log = np.log(acled_quarterly_adm0_pivot+1)
acled_quarterly_adm0_log.to_csv("../data/TB012_ACLED_QUARTERLY_ADM0_LOG.csv", index=True)
acled_quarterly_adm0_log

,NUMBER_EVENTS_ABW,NUMBER_EVENTS_AFG,NUMBER_EVENTS_AGO,NUMBER_EVENTS_AIA,NUMBER_EVENTS_ALA,NUMBER_EVENTS_ALB,NUMBER_EVENTS_AND,NUMBER_EVENTS_ARE,NUMBER_EVENTS_ARG,NUMBER_EVENTS_ARM,...,SUM(FATALITIES_STRA)_WSM,SUM(FATALITIES_STRA)_XAR,SUM(FATALITIES_STRA)_XIT,SUM(FATALITIES_STRA)_XKK,SUM(FATALITIES_STRA)_XKO,SUM(FATALITIES_STRA)_XNC,SUM(FATALITIES_STRA)_YEM,SUM(FATALITIES_STRA)_ZAF,SUM(FATALITIES_STRA)_ZMB,SUM(FATALITIES_STRA)_ZWE
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,3.218876,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-04-01,NaN,NaN,4.510860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-07-01,NaN,NaN,4.770685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1997-10-01,NaN,NaN,3.496508,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
1998-01-01,NaN,NaN,3.637586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01,0.000000,8.041413,3.871201,0.693147,0.000000,3.583519,1.386294,0.000000,6.758095,4.934474,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2021-04-01,0.000000,8.332549,3.295837,0.000000,0.000000,4.454347,0.693147,0.000000,6.722630,4.919981,...,0.0,0.0,0.0,0.0,0.0,0.0,3.258097,0.0,0.0,0.0
2021-07-01,0.693147,7.549609,3.688879,0.000000,0.000000,3.258097,0.693147,0.693147,6.458338,4.890349,...,0.0,0.0,0.0,0.0,0.0,0.0,1.791759,0.0,0.0,0.0


### ADM1 QUARTERLY

In [24]:
acled_quarterly_adm1_pivot = acled_monthly_adm1_pivot.resample('3MS').apply(lambda x: x.sum(skipna=False))
# log all values to enable later log change calculation by difference
acled_quarterly_adm1_log = np.log(acled_quarterly_adm1_pivot+1)
acled_quarterly_adm1_log.to_csv("../data/TB013_ACLED_QUARTERLY_ADM1_LOG.csv", index=True)
acled_quarterly_adm1_log

,NUMBER_EVENTS_ABW.1_1,NUMBER_EVENTS_AFG.10_1,NUMBER_EVENTS_AFG.11_1,NUMBER_EVENTS_AFG.12_1,NUMBER_EVENTS_AFG.13_1,NUMBER_EVENTS_AFG.14_1,NUMBER_EVENTS_AFG.15_1,NUMBER_EVENTS_AFG.16_1,NUMBER_EVENTS_AFG.17_1,NUMBER_EVENTS_AFG.18_1,...,SUM(FATALITIES_STRA)_ZWE.10_1,SUM(FATALITIES_STRA)_ZWE.1_1,SUM(FATALITIES_STRA)_ZWE.2_1,SUM(FATALITIES_STRA)_ZWE.3_1,SUM(FATALITIES_STRA)_ZWE.4_1,SUM(FATALITIES_STRA)_ZWE.5_1,SUM(FATALITIES_STRA)_ZWE.6_1,SUM(FATALITIES_STRA)_ZWE.7_1,SUM(FATALITIES_STRA)_ZWE.8_1,SUM(FATALITIES_STRA)_ZWE.9_1
EVENT_DATE_MONTH,,,,,,,,,,,,,,,,,,,,,
1997-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-04-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-07-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1997-10-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-01-01,0.000000,NaN,0.0000,NaN,NaN,6.962243,0.000000,NaN,NaN,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN
2021-04-01,0.000000,NaN,7.3518,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,...,0.0,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2021-07-01,0.693147,NaN,NaN,NaN,NaN,0.000000,7.133296,NaN,NaN,NaN,...,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN
